In [3]:
import pandas as pd

In [4]:
import datetime

In [5]:
pd.__version__

'2.2.2'

In [6]:
pip install pyarrow


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [8]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
import sklearn

In [10]:
sklearn.__version__

'1.5.1'

In [11]:
len(df.columns)

19

In [12]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
print(df['duration'].std())

42.59435124195458


In [13]:
# prompt: Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
# What fraction of the records left after you dropped the outliers?

import pandas as pd

# Filter the DataFrame to include durations between 1 and 60 minutes
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records remaining
fraction_remaining = df_filtered.shape[0] / df.shape[0]

print(f"Fraction of records remaining after removing outliers: {fraction_remaining:.2f}")

Fraction of records remaining after removing outliers: 0.98


In [14]:
# prompt: Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

from sklearn.feature_extraction import DictVectorizer

# Turn the dataframe into a list of dictionaries
categorical = ['PULocationID', 'DOLocationID']
df_filtered[categorical] = df_filtered[categorical].astype(str)
train_dicts = df_filtered[categorical].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Get a feature matrix from it
# X_train is already the feature matrix

# What's the dimensionality of this matrix (number of columns)?
num_columns = X_train.shape[1]
print(f"Dimensionality of the feature matrix: {num_columns}")

/tmp/ipykernel_24037/2531235375.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[categorical] = df_filtered[categorical].astype(str)


Dimensionality of the feature matrix: 515


In [15]:
# prompt: Now let's use the feature matrix from the previous step to train a model.
# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
lr = LinearRegression()
lr.fit(X_train, df_filtered['duration'])

# Predict on the training data
y_pred_train = lr.predict(X_train)

# Calculate the RMSE on the training data
rmse_train = mean_squared_error(df_filtered['duration'], y_pred_train, squared=False)

print(f"RMSE on train: {rmse_train:.2f}")


RMSE on train: 7.65


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
df =pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
# prompt: Now let's apply this model to the validation dataset (February 2023).
# What's the RMSE on validation?

# Read the validation dataset
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# Preprocess the validation data
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')

# Transform the validation data using the fitted DictVectorizer
X_val = dv.transform(val_dicts)

# Predict on the validation data
y_pred_val = lr.predict(X_val)

# Calculate the RMSE on the validation data
rmse_val = mean_squared_error(df_val['duration'], y_pred_val, squared=False)

print(f"RMSE on validation: {rmse_val:.2f}")